### Import Dependencies

In [1]:
import random
random.seed(0)

In [2]:
import torch
import torch.nn as nn
import torchvision
from torchvision import transforms

import optuna

### Constant Variables

In [3]:
DATASET_PATH = './knee-osteoarthritis_2'

TRAIN_PATH = f'{DATASET_PATH}/train'
VAL_PATH = f'{DATASET_PATH}/val'
TEST_PATH = f'{DATASET_PATH}/test'

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

print(device)

In [5]:
classes = range(4)

### Dataset Preparation

In [6]:
from src.dataset.augmented_dataset import get_KneeOsteoarthritis_Edges, KneeOsteoarthritis_Edges

transform_toTensor = transforms.Compose([transforms.ToTensor()])

train_dataset = torchvision.datasets.ImageFolder(TRAIN_PATH, transform_toTensor)
val_dataset = torchvision.datasets.ImageFolder(VAL_PATH, transform_toTensor)
test_dataset = torchvision.datasets.ImageFolder(TEST_PATH, transform_toTensor)

dataset_all = torch.utils.data.ConcatDataset([train_dataset, val_dataset, test_dataset])

train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(dataset_all, [0.7, 0.1, 0.2])

train_dataset = KneeOsteoarthritis_Edges(train_dataset)
val_dataset = KneeOsteoarthritis_Edges(val_dataset)
test_dataset = KneeOsteoarthritis_Edges(test_dataset)

In [ ]:
print(len(train_dataset), len(val_dataset), len(test_dataset))

### Data Loaders

In [8]:
from torch.utils.data import DataLoader

BATCH_SIZE = 128

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=True)

## Optuna

In [9]:
def define_model(trial, layer_size, dropout_main):
    class IntermediarySpaceModel(nn.Module):
        def __init__(self, train_device, num_classes: int = 5, dropout: float = 0.5) -> None:
            super().__init__()
            
            # Size of layer block
            
            S = layer_size
            # S = trial.suggest_int("layer_size", 12, 24)
            self.train_device = train_device
            
            # Images
            self.imagesClassifier = nn.Sequential(
                nn.Conv2d(3, S*2, kernel_size=11, stride=4, padding=2),
                nn.ReLU(inplace=True),
                nn.MaxPool2d(kernel_size=3, stride=2),
                nn.Dropout(p=dropout*0.2),
                nn.Conv2d(S*2, S*2, kernel_size=5, padding=2),
                nn.ReLU(inplace=True),
                nn.MaxPool2d(kernel_size=3, stride=2),
                nn.Dropout(p=dropout*0.4),
                nn.Conv2d(S*2, S*2, kernel_size=3, padding=1),
                nn.ReLU(inplace=True),
                nn.Dropout(p=dropout*0.6),
                nn.Conv2d(S*2, S, kernel_size=3, padding=1),
                nn.ReLU(inplace=True),
                nn.MaxPool2d(kernel_size=3, stride=2),
                
                nn.Flatten(),
                nn.Dropout(p=dropout*0.8),
                nn.Linear(S * 7 * 7, S*2),
            )

            self.edgesClassifier = nn.Sequential(
                nn.Conv2d(1, S*2, kernel_size=11, stride=4, padding=2),
                nn.ReLU(inplace=True),
                nn.MaxPool2d(kernel_size=3, stride=2),
                nn.Dropout(p=dropout*0.4),
                nn.Conv2d(S*2, S*2, kernel_size=5, padding=2),
                nn.ReLU(inplace=True),
                nn.MaxPool2d(kernel_size=3, stride=2),
                nn.Dropout(p=dropout*0.6),
                nn.Conv2d(S*2, S, kernel_size=3, padding=1),
                nn.ReLU(inplace=True),
                nn.MaxPool2d(kernel_size=3, stride=2),
                
                nn.Flatten(),
                nn.Dropout(p=dropout*0.8),
                nn.Linear(S * 6 * 6, S*2),
            )
            
            self.outputCombiner = nn.Sequential(
                nn.ReLU(inplace=True),
                nn.Dropout(p=dropout),
                nn.Linear(S*4, S*3),
                nn.ReLU(inplace=True),
                nn.Dropout(p=dropout),
                nn.Linear(S*3, S),
                nn.ReLU(inplace=True),
                nn.Dropout(p=dropout),
                nn.Linear(S, num_classes),
            )
            
        def forward(self, data: tuple) -> torch.Tensor:
            
            images, edges = data
            
            images = images.to(self.train_device)
            edges = edges.to(self.train_device)
            
            # Images
            images = self.imagesClassifier(images)
            
            # Edges
            edges = self.edgesClassifier(edges)
            
            # Combining outputs
            concated = torch.cat((images, edges), 1)
            res = self.outputCombiner(concated)
            
            return res

    # DROPOUT_MAIN = trial.suggest_float("dropout_main", 0.3, 0.7)
    model = IntermediarySpaceModel(device, 4, dropout_main)
    model = model.to(device)
    
    return model


In [ ]:
import torch.optim as optim
from src.other import get_classes_frequencies
    
class_weights = get_classes_frequencies(train_dataset)
weights_tensor = torch.Tensor(list(class_weights.values())).to(device)
print(class_weights, weights_tensor)

criterion = nn.CrossEntropyLoss(weights_tensor)

In [11]:
EXP_NAME = "confusion_matrix_test"

from torch.utils.tensorboard import SummaryWriter

# logger = SummaryWriter(log_dir=f"logs/test/{EXP_NAME}")
logger = SummaryWriter(log_dir=f"logs/test")

In [12]:
from src.training import Trainer
    
def objective(trial):
    # Hyperparameters
    layer_size = trial.suggest_int("layer_size", 8, 28)
    dropout_main = trial.suggest_float("dropout_main", 0.3, 0.7)
    initial_lr = 0.001
    lr_decay_rate_100 = trial.suggest_float("decay_rate_100", 0.3, 0.8)
    reg_type = "L2"
    reg_lambda = trial.suggest_float("reg_lambda", 0.0001, 0.1)
    
    hparams = {
      "layer_size": layer_size,
      "dropout_main": dropout_main,
      "initial_lr": initial_lr,
      "lr_decay_rate_100": lr_decay_rate_100,
      "reg_type": reg_type,
      "reg_lambda": reg_lambda,
    }
    
    # Model
    model = define_model(trial, layer_size, dropout_main)
    
    # Optimizer
    optimizer = optim.Adam(model.parameters(), lr=initial_lr)
    
    # lr scheduler
    lr_decay_rate_1 = lr_decay_rate_100**(1/100)
    lr_scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer=optimizer, gamma=lr_decay_rate_1)
    
    # Trainer object
    trainer = Trainer(
      model=model,
      classes=classes,
      train_loader=train_loader,
      val_loader=val_loader,
      criterion=criterion,
      optimizer=optimizer,
      device=device,
      lr_scheduler=lr_scheduler,
      reg_type=reg_type,
      reg_lambda=0.01,
      # tensorboard_logger=logger
    )

        
    # Training loop
    tAccuracy, tLoss, vAccuracy, vLoss = 0, 0, 0, 0
    
    val_accuracy_midpoint = 60
    relDifference = None
    
    for step in range(40):
      tAccuracy, tLoss, vAccuracy, vLoss, _ = trainer.train_single()
      difference = tAccuracy - vAccuracy
      relDifference = difference * (vAccuracy-val_accuracy_midpoint) / 100

      trial.report(relDifference, step)
      
      if trial.should_prune():
        print(f'Training: accuracy: {tAccuracy:.3f}%, Validation: accuracy: {vAccuracy:.3f}%, relDifference: {relDifference:.3f}')
        raise optuna.TrialPruned()
    
    print(f'Training: accuracy: {tAccuracy:.3f}%, Validation: accuracy: {vAccuracy:.3f}%')
    
    logger.add_hparams(hparams, {'hparam/tAccuracy': tAccuracy, 'hparam/vAccuracy': vAccuracy, 'relDifference': relDifference}, run_name=f"Trial_nr_{trial.number}")
    
    return relDifference

In [ ]:
from optuna.samplers import TPESampler

pruner = optuna.pruners.MedianPruner()
sampler = TPESampler()

study = optuna.create_study(directions=["maximize"], sampler=sampler, pruner=pruner)
study.optimize(objective, n_trials=120)

print("Number of finished trials: ", len(study.trials))